# Evaluation Analysis: Banking Agent Performance

This notebook analyzes the performance of the fine-tuned banking agent model.

In [ ]:
import sys
sys.path.append('../src')

import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from evaluation import BankingAgentEvaluator
import numpy as np

## Load Evaluation Results

In [ ]:
# Load evaluation results (if available)
results_path = "../results/evaluation_results.json"

try:
    with open(results_path, 'r') as f:
        results = json.load(f)
    
    print("Evaluation results loaded successfully!")
    print(f"Overall AC Score: {results['overall_ac_score']:.3f}")
    print(f"Overall TSQ Score: {results['overall_tsq_score']:.3f}")
    print(f"Total Scenarios: {results['total_scenarios']}")
    
    # Convert to DataFrame for analysis
    df = pd.DataFrame(results['detailed_results'])
    
except FileNotFoundError:
    print("Evaluation results not found. Run evaluation first.")
    print("Use: python src/evaluation.py")
    
    # Create sample data for demonstration
    np.random.seed(42)
    n_samples = 20
    
    sample_data = {
        'scenario_id': range(n_samples),
        'persona_index': np.random.randint(0, 100, n_samples),
        'ac_score': np.random.beta(2, 1, n_samples),  # Skewed towards higher scores
        'tsq_score': np.random.beta(1.5, 1, n_samples),
        'completed_goals': np.random.randint(3, 8, n_samples),
        'total_goals': np.random.randint(6, 9, n_samples)
    }
    
    df = pd.DataFrame(sample_data)
    df['ac_score'] = df['completed_goals'] / df['total_goals']  # Ensure consistency
    
    results = {
        'overall_ac_score': df['ac_score'].mean(),
        'overall_tsq_score': df['tsq_score'].mean(),
        'total_scenarios': len(df)
    }
    
    print("Using sample data for demonstration:")
    print(f"Sample AC Score: {results['overall_ac_score']:.3f}")
    print(f"Sample TSQ Score: {results['overall_tsq_score']:.3f}")

## Performance Analysis

In [ ]:
# Performance distribution analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# AC Score distribution
axes[0, 0].hist(df['ac_score'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].axvline(df['ac_score'].mean(), color='red', linestyle='--', label=f'Mean: {df["ac_score"].mean():.3f}')
axes[0, 0].set_xlabel('Action Completion Score')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Distribution of AC Scores')
axes[0, 0].legend()

# TSQ Score distribution
axes[0, 1].hist(df['tsq_score'], bins=20, alpha=0.7, color='lightgreen', edgecolor='black')
axes[0, 1].axvline(df['tsq_score'].mean(), color='red', linestyle='--', label=f'Mean: {df["tsq_score"].mean():.3f}')
axes[0, 1].set_xlabel('Tool Selection Quality Score')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Distribution of TSQ Scores')
axes[0, 1].legend()

# Goals completion analysis
goal_completion_rate = df['completed_goals'] / df['total_goals']
axes[1, 0].scatter(df['total_goals'], df['completed_goals'], alpha=0.6)
axes[1, 0].plot([df['total_goals'].min(), df['total_goals'].max()], 
                [df['total_goals'].min(), df['total_goals'].max()], 
                'r--', label='Perfect completion')
axes[1, 0].set_xlabel('Total Goals')
axes[1, 0].set_ylabel('Completed Goals')
axes[1, 0].set_title('Goal Completion Analysis')
axes[1, 0].legend()

# Score correlation
axes[1, 1].scatter(df['ac_score'], df['tsq_score'], alpha=0.6)
axes[1, 1].set_xlabel('AC Score')
axes[1, 1].set_ylabel('TSQ Score')
axes[1, 1].set_title('AC vs TSQ Score Correlation')

# Add correlation coefficient
correlation = df['ac_score'].corr(df['tsq_score'])
axes[1, 1].text(0.05, 0.95, f'Correlation: {correlation:.3f}', 
                transform=axes[1, 1].transAxes, bbox=dict(boxstyle='round', facecolor='white'))

plt.tight_layout()
plt.show()

## Performance by Persona Analysis

In [ ]:
# Group by persona and analyze performance
persona_performance = df.groupby('persona_index').agg({
    'ac_score': ['mean', 'std', 'count'],
    'tsq_score': ['mean', 'std'],
    'completed_goals': 'sum',
    'total_goals': 'sum'
}).round(3)

# Flatten column names
persona_performance.columns = ['_'.join(col).strip() for col in persona_performance.columns]

# Show top and bottom performing personas
print("Top 5 Performing Personas (by AC Score):")
top_personas = persona_performance.nlargest(5, 'ac_score_mean')
print(top_personas[['ac_score_mean', 'tsq_score_mean', 'ac_score_count']])

print("\nBottom 5 Performing Personas (by AC Score):")
bottom_personas = persona_performance.nsmallest(5, 'ac_score_mean')
print(bottom_personas[['ac_score_mean', 'tsq_score_mean', 'ac_score_count']])

## Error Analysis

In [ ]:
# Identify scenarios with low performance
low_performance_threshold = 0.5

low_ac_scenarios = df[df['ac_score'] < low_performance_threshold]
low_tsq_scenarios = df[df['tsq_score'] < low_performance_threshold]

print(f"Scenarios with AC Score < {low_performance_threshold}: {len(low_ac_scenarios)}")
print(f"Scenarios with TSQ Score < {low_performance_threshold}: {len(low_tsq_scenarios)}")

if len(low_ac_scenarios) > 0:
    print("\nLow AC Score Scenarios:")
    print(low_ac_scenarios[['scenario_id', 'ac_score', 'completed_goals', 'total_goals']].head())

# Performance by goal complexity
df['goal_complexity'] = pd.cut(df['total_goals'], bins=[0, 6, 7, 10], labels=['Simple', 'Medium', 'Complex'])

complexity_performance = df.groupby('goal_complexity').agg({
    'ac_score': ['mean', 'std'],
    'tsq_score': ['mean', 'std'],
    'scenario_id': 'count'
}).round(3)

print("\nPerformance by Goal Complexity:")
print(complexity_performance)

## Benchmark Comparison

In [ ]:
# Compare with baseline performance (hypothetical)
baseline_scores = {
    'Base xLAM-2-1b': {'AC': 0.65, 'TSQ': 0.70},
    'Fine-tuned Model': {'AC': results['overall_ac_score'], 'TSQ': results['overall_tsq_score']},
    'GPT-4 (Reference)': {'AC': 0.85, 'TSQ': 0.88},
    'Claude-3.5 (Reference)': {'AC': 0.87, 'TSQ': 0.90}
}

# Create comparison chart
models = list(baseline_scores.keys())
ac_scores = [baseline_scores[model]['AC'] for model in models]
tsq_scores = [baseline_scores[model]['TSQ'] for model in models]

x = np.arange(len(models))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))
bars1 = ax.bar(x - width/2, ac_scores, width, label='AC Score', alpha=0.8)
bars2 = ax.bar(x + width/2, tsq_scores, width, label='TSQ Score', alpha=0.8)

ax.set_xlabel('Models')
ax.set_ylabel('Score')
ax.set_title('Model Performance Comparison')
ax.set_xticks(x)
ax.set_xticklabels(models, rotation=45, ha='right')
ax.legend()
ax.set_ylim(0, 1)

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.3f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Calculate improvement
base_ac = baseline_scores['Base xLAM-2-1b']['AC']
fine_tuned_ac = baseline_scores['Fine-tuned Model']['AC']
improvement = ((fine_tuned_ac - base_ac) / base_ac) * 100

print(f"\nImprovement over base model:")
print(f"AC Score improvement: {improvement:.1f}%")
print(f"Absolute AC improvement: {fine_tuned_ac - base_ac:.3f}")

## Summary and Recommendations

In [ ]:
print("=" * 60)
print("EVALUATION SUMMARY")
print("=" * 60)

print(f"Overall Performance:")
print(f"  • Action Completion (AC): {results['overall_ac_score']:.3f}")
print(f"  • Tool Selection Quality (TSQ): {results['overall_tsq_score']:.3f}")
print(f"  • Total Scenarios Evaluated: {results['total_scenarios']}")

print(f"\nPerformance Distribution:")
print(f"  • AC Score Std Dev: {df['ac_score'].std():.3f}")
print(f"  • TSQ Score Std Dev: {df['tsq_score'].std():.3f}")
print(f"  • High Performers (AC > 0.8): {len(df[df['ac_score'] > 0.8])} scenarios")
print(f"  • Low Performers (AC < 0.5): {len(df[df['ac_score'] < 0.5])} scenarios")

print(f"\nRecommendations for Improvement:")
print(f"  1. Focus on complex scenarios (7+ goals) - lower performance observed")
print(f"  2. Improve tool selection consistency - TSQ variance is high")
print(f"  3. Analyze low-performing personas for training data augmentation")
print(f"  4. Consider multi-turn conversation fine-tuning")
print(f"  5. Implement more sophisticated evaluation metrics")

print(f"\nNext Steps:")
print(f"  • Deploy model for A/B testing")
print(f"  • Collect real user feedback")
print(f"  • Iterate on training data quality")
print(f"  • Consider ensemble approaches")

print("=" * 60)